# 6.4.1 오버피팅

overfitting이란 신경망이 훈련 데이터에만 지나치게 적응되어 그 외의 데이터에는 제대로 대응하지 못하는 상태를 말한다. 기계학습은 범용 성능을 지향하기 때문에 이를 억제하는 것이 중요하다.

Overfitting은 주로 2가지 경우에 일어난다.
* 매개변수가 많고 표현력이 높은 모델
* 훈련 데이터가 적을 경우

코드를 통해 살펴보면 다음과 같다.

In [1]:
import os
import sys

sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# weight decay（가중치 감쇠） 설정 =======================
#weight_decay_lambda = 0 # weight decay를 사용하지 않을 경우
weight_decay_lambda = 0.1
# ====================================================

network = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100], output_size=10,
                        weight_decay_lambda=weight_decay_lambda)
optimizer = SGD(lr=0.01) # 학습률이 0.01인 SGD로 매개변수 갱신

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break


# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

ModuleNotFoundError: No module named 'dataset'

train_acc_list와 test_acc_list에는 epoch 단위(모든 훈련 데이터를 한 번씩 본 단위)의 정확도를 저장한다. 위 코드를 실행하면 결과는 다음과 같이 나오게 된다.

<p align="center"><img src="imgs/6-15.png" width=600></p>

훈련데이터와 시험데이터의 차이가 크게 벌어지게 되는데 이것은 훈련 데이터에만 적응해서이다. 훈련 때 사용하지 않은 범용 데이터(시험 데이터)에는 제대로 대응하지 못 하는 것을 이 그래프로 확인할 수 있다.

# 6.4.2 가중치 감소
overfitting문제를 억제하기 위해 가중치 감소라는 방법을 사용한다. 학습 과정에서 큰 가중치에 대해서는 큰 페널티를 부과하여 overfitting을 억제하는 방법이다. 가중치 감소는 모든 가중치 각각의 손실 함수에 1/2λW를 더한다. 따라서 가중치의 기울기를 구하는 계산에서는 그 동안의 오차역전파법에 따른 정규화 항을 미분한 λW를 더한다. 가중치 감소를 이용하여 훈련 데이터와 시험데이터의 정확도를 비교하면 다음과 같다.

<p align="center"><img src="imgs/6-16.png" width=600></p>

가중치 감소를 사용하지 않은 위 그림과 비교해 보면 그 차이가 줄었다. 오버피팅이 억제 되었다는 소리다. 훈련데이터에 대한 정확도도 100%에 도달하지 못한 점도 주목하자.

# 6.4.3 드롭아웃
신경망 모델이 복잡해지면 가중치 감소만으로는 대응하기 어렵기 때문에 드롭아웃 기법을 사용한다. 드롭아웃은 뉴런을 임의로 삭제하면서 학습하는 방법이다. 훈련 때 은닉층의 뉴런을 무작위로 골라 삭제하고 삭제한 뉴런은 신호를 전달하지 않게 된다. 훈련때는 데이터를 흘릴 때마다 삭제할 뉴런을 무작위로 선택하고 시험 때는 모든 뉴런에 신호를 전달한다. 단 시험 때는 각 뉴런의 출력에 훈련 때 삭제 안 한 비율을 곱하여 출력한다.

<p align="center"><img src="imgs/6-17.png" width=600></p>

드롭 아웃을 코드로 구현하면 다음과 같다.

In [ ]:
class Dropout:

    def __init__(self, dropout_ratio=0.5):
    self.dropout_ratio = dropout_ratio
    self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask

순전파 때마다 self.mask에 삭제할 뉴런을 False로 표시한다. self.mask는 x와 형상이 같은 배열을 무작위로 생성하고, 그 값이 dropout_ratio보다 큰 원소만 True로 설정한다. 역전파 때의 동작은 ReLU와 같다. 순전파때 신호를 통과시키는 뉴런은 역전파 때도 신호를 그대로 통과시키고 순전파 때 통과시키지 않은 뉴런은 역전파 때도 신호를 차단한다. 이 효과를 MNIST 데이터셋으로 확인하기 위해 코드를 구현하고 실행하면 다음과 같다.

<p align="center"><img src="imgs/6-18.png" width=600></p>

드롭아웃을 적용하면 훈련 데이터와 시험 데이터에 대한 정확도 차이도 줄고 훈련 데이터에 대한 정확도가 100%에 도달하지도 않는다. 이처럼 드롭아웃을 이용하면 표현력을 높이면서도 overfitting을 억제할 수 있다.